In [73]:
import sys, os, multiprocessing, urllib, csv
from PIL import Image
from io import BytesIO


def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:2] for line in csvreader][1:]
    return key_url_list


def DownloadImage(key_url):
    out_dir = os.path.join(os.getcwd(), 'output')

    (key, url) = key_url
    filename = os.path.join(out_dir, '%s.jpg' % key)

    if os.path.exists(filename):
        print('Image %s already exists. Skipping download.' % filename)
        return

    try:
        response = urllib.request.urlopen(url)
        image_data = response.read()
    except:
        print('Warning: Could not download image %s from %s' % (key, url))
        return

    try:
        pil_image = Image.open(BytesIO(image_data))
        pil_image = pil_image.resize((800, 800))
    except:
        print('Warning: Failed to parse image %s' % key)
        return


    try:
        pil_image.save(filename, format='JPEG', quality=90)
    except:
        print('Warning: Failed to save image %s' % filename)
        return


def Run():
    data_file = os.path.join(os.getcwd(), os.path.join('landmark-recognition-challenge', 'train.csv'))
    out_dir = os.path.join(os.getcwd(), 'output')

    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    key_url_list = ParseData(data_file)[:3200]
    pool = multiprocessing.Pool(processes=6)
    pool.map(DownloadImage, key_url_list)


#if __name__ == '__main__':
#    Run()


In [74]:
import pandas as pd
import numpy as np

In [75]:
a = pd.read_csv('landmark-recognition-challenge/train.csv')
OUTPUT_DIR  = 'output/'


In [76]:
import keras

In [57]:
arr_list = []
for file in os.listdir(OUTPUT_DIR):
    try:
        image = tf.keras.preprocessing.image.load_img(OUTPUT_DIR + file)
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_arr = np.array([input_arr])  # Convert single image to a batch.
        inpur_arr = np.expand_dims(intput_arr, axis=0)  # or axis=1

        arr_list.append(input_arr)
    except:
        pass
    #predictions = model.predict(input_arr)


In [60]:
input_arr.shape

(1, 150, 150, 3)

In [ ]:
from skimage import io
io.imshow(input_arr)

In [47]:
np.hstack(input_arr).shape

(150, 150, 3)

In [20]:
id_list = []
for file in os.listdir('output'):
    file_name, ext = os.path.splitext(file)
    id_list.append(file_name)

In [21]:
train_data = a[a.id.isin(id_list)]

In [22]:
from sklearn.model_selection import train_test_split

/Users/jonathanreich/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/jonathanreich/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/jonathanreich/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [268]:
def convert_image_to_tensor(img):
    raw_img = np.asarray(img)
    raw_img = raw_img[None, ...]
    return raw_img

In [343]:
def create_x_y(output_dir):
    from skimage import io
    a = pd.read_csv('landmark-recognition-challenge/train.csv')
    X = []
    Y = []
    for file in os.listdir(output_dir):
        try:
            img = io.imread(output_dir + file)
            file, ext = os.path.splitext(file)
            data = a[a.id==file]
            label = data['landmark_id'].values.tolist()
            #img = img[None, ...]
            X.append(img)
            Y.append(label)
        except:
            pass
    return X, Y

In [102]:
def create(OUTPUT_DIR):
    a = pd.read_csv('landmark-recognition-challenge/train.csv')
    Y = []
    arr_list = []
    for file in os.listdir(OUTPUT_DIR):
        try:
            image = tf.keras.preprocessing.image.load_img(OUTPUT_DIR + file)
            file, ext = os.path.splitext(file)
            data = a[a.id==file]
            print(data)
            label = data['landmark_id'].values.tolist()
            print(label)
            y.append(label)
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr])  # Convert single image to a batch.
            inpur_arr = np.expand_dims(intput_arr, axis=0)  # or axis=1
            arr_list.append(input_arr)
        except:
            pass
        #predictions = model.predict(input_arr)
    return arr_list, Y

In [104]:
X,Y = create('output/')

                    id                                                url  landmark_id
2842  81fbd78fb1459162  http://lh5.ggpht.com/-Rb8X-HxoNOA/UAPXhC4lDOI/...         5132
[5132]
                    id                                                url  landmark_id
2834  2d1b4cd04cf0e2b4  https://lh3.googleusercontent.com/-tofYqjNwYWE...        14062
[14062]
                   id                                                url  landmark_id
922  1cb4ae7f83b55fd8  https://lh3.googleusercontent.com/-QlgEDzfyYe4...        10975
[10975]
                   id                                                url  landmark_id
224  7e79461955afde5b  https://lh4.googleusercontent.com/-1NKiIkVjzo0...        10369
[10369]
                    id                                                url  landmark_id
2283  618840ec4d76ee3e  http://lh3.ggpht.com/-oBhXVlIxtxU/STLiM1YtyuI/...         2436
[2436]
                    id                                                url  landmark_id
2863  018

                   id                                                url  landmark_id
294  3e411f690e1c59be  https://lh4.googleusercontent.com/-YJ352Qg-WWk...         6651
[6651]
                    id                                                url  landmark_id
2396  5af8d920af53d746  https://lh5.googleusercontent.com/-zL--qRWap9c...        13132
[13132]
                   id                                                url  landmark_id
442  bb74024936003e18  http://lh3.ggpht.com/-nxFaG0LfTvI/Ss-AfVTaxZI/...        12867
[12867]
                    id                                                url  landmark_id
2720  cd3d8e16854651f7  https://lh6.googleusercontent.com/-Osf0UdZGFYo...         8429
[8429]
                   id                                                url  landmark_id
236  49c27d0187fd72de  http://lh3.ggpht.com/-UixaFtbWCCk/SMwJnEuazYI/...         9086
[9086]
                   id                                                url  landmark_id
976  090900cd

                    id                                                url  landmark_id
3172  7b878cb2a39bf6d6  https://lh4.googleusercontent.com/-OK48bh5WHyA...        10577
[10577]
                    id                                                url  landmark_id
1032  a090d2e77bab5682  https://lh5.googleusercontent.com/-zFRyS20hG4g...        13212
[13212]
                    id                                                url  landmark_id
1097  d44aa4cf9d7dc423  http://lh4.ggpht.com/-zR4WQaaE6Yc/SkzC6KD7L2I/...         8029
[8029]
                    id                                                url  landmark_id
1874  9096c8d4e11bfbef  https://lh6.googleusercontent.com/-bTTKO3Baonc...         9633
[9633]
                  id                                                url  landmark_id
86  2faffd06a149ee77  https://lh3.googleusercontent.com/-EpNtXQntMr0...         5554
[5554]
                    id                                                url  landmark_id
2990  4d95

                    id                                                url  landmark_id
2378  039f175165e31f4c  http://lh3.ggpht.com/-sa0PEZkgXnA/SP2uELe9NgI/...         8054
[8054]
                   id                                                url  landmark_id
108  4a89b5d0b05218e5  https://lh5.googleusercontent.com/-LTLk7cTzaJE...        10045
[10045]
                   id                                                url  landmark_id
790  07414e2ae8a5810c  https://lh5.googleusercontent.com/-hQobB3glHCw...          233
[233]
                    id                                                url  landmark_id
2590  89308574baea6563  https://lh6.googleusercontent.com/-CuxrV6RrhG8...         1135
[1135]
                    id                                                url  landmark_id
1783  4fec70a2712df5a9  http://lh4.ggpht.com/-ggrnhQa2F5c/Ty9KUpnOMII/...         6051
[6051]
                    id                                                url  landmark_id
3102  e4ad51

                    id                                                url  landmark_id
2592  2a4802e8bc9de537  https://lh4.googleusercontent.com/-9tgrryCJLsE...         6168
[6168]
                    id                                                url  landmark_id
1397  11533a898cb488ed  https://lh6.googleusercontent.com/-aTXAe9OBKSw...          542
[542]
                  id                                                url  landmark_id
28  d21e1c3455134c0d  https://lh6.googleusercontent.com/-zBjqt9mKxfA...         3457
[3457]
                    id                                                url  landmark_id
1567  dffbf7014cd4cc37  https://lh6.googleusercontent.com/-9hl_ivyXrK0...         5474
[5474]
                   id                                                url  landmark_id
214  2bbd6c377726e257  https://lh5.googleusercontent.com/-57qKqR5r9n4...         2606
[2606]
                   id                                                url  landmark_id
287  773ae8d63ba

...

                    id                                                url  landmark_id
2271  70666ea4741c4fc4  https://lh5.googleusercontent.com/-TgkmF-Pfo5o...         1582
[1582]
                    id                                                url  landmark_id
2782  3dec7140fad424b2  http://lh6.ggpht.com/-lCiZ-Jwj6LA/R3-9GWp5JlI/...          538
[538]
                    id                                                url  landmark_id
2792  438be85c923e6506  https://lh3.googleusercontent.com/-bGwN27Cp5Zs...         6241
[6241]
                    id                                                url  landmark_id
1442  41b59f22429f5221  https://lh6.googleusercontent.com/-OrATVpAXzuo...        11535
[11535]
                    id                                                url  landmark_id
1797  def819167d5ed760  https://lh4.googleusercontent.com/-pp-qiuRkrQE...         4829
[4829]
                    id                                                url  landmark_id
1962  84

                    id                                                url  landmark_id
1709  ffb60650adf0cdfa  https://lh3.googleusercontent.com/-LIq6Drsbd3A...         7172
[7172]
                   id                                                url  landmark_id
779  a4090eeb94d12254  http://lh4.ggpht.com/-xuoUT46N34U/SHuhRUy37lI/...         9633
[9633]
                    id                                                url  landmark_id
1534  142262f47457407c  https://lh3.googleusercontent.com/-0iVaX8ABAno...          912
[912]
                    id                                                url  landmark_id
2471  ac8b5ad556548e1c  http://lh3.ggpht.com/-3j8-gxlmd64/SQJjIT7re3I/...        12618
[12618]
                    id                                                url  landmark_id
2784  65d41ba7a709dcf6  http://lh4.ggpht.com/-9d6U3zw1mww/Sth1_HhTJ1I/...         6397
[6397]
                    id                                                url  landmark_id
3104  fb14

                    id                                                url  landmark_id
2383  d35ff0c683886a29  https://lh6.googleusercontent.com/-a-tExLPgMQU...         7648
[7648]
                    id                                                url  landmark_id
3086  b454763a93fc30ba  https://lh6.googleusercontent.com/-443NU1rm2VM...         3278
[3278]
                   id                                                url  landmark_id
153  4872c66e96574856  http://lh6.ggpht.com/-Kuf0XpZ7w1k/ThdGp6rjlqI/...         5166
[5166]
                   id                                                url  landmark_id
424  5e0ed811a1954e96  http://lh4.ggpht.com/-K2jURg8VWj4/Te4RfwD6oXI/...         6241
[6241]
                   id                                                url  landmark_id
536  b07810cac1d5f397  https://lh6.googleusercontent.com/-TUVw6M2Yn2M...         3845
[3845]
                    id                                                url  landmark_id
2625  358bde38

                    id                                                url  landmark_id
1055  df64df4750d090d5  https://lh3.googleusercontent.com/-J5paR-RREbY...        10936
[10936]
                   id                                                url  landmark_id
770  57c73b8685852981  http://lh3.ggpht.com/-f5466omiui4/S5dud5OYRJI/...         9358
[9358]
                    id                                                url  landmark_id
2467  9e4390bea64d5c4a  https://lh5.googleusercontent.com/-nQpjwNE2zMI...        10175
[10175]
                    id                                                url  landmark_id
3005  0c14467ec3c11aba  http://lh6.ggpht.com/-VGNJMUfAkTY/Rrhir3XCvmI/...         4987
[4987]
                    id                                                url  landmark_id
1220  0ecb862b0b2df4e8  https://lh3.googleusercontent.com/-wuvrDhM7D3w...         9215
[9215]
                    id                                                url  landmark_id
1547  cc

                    id                                                url  landmark_id
1076  e05328e43556af81  http://lh5.ggpht.com/-1v0djvPBIZo/SqqIoU31NFI/...         1256
[1256]
                    id                                                url  landmark_id
1447  1ef0d7871f238b6c  http://lh5.ggpht.com/-eixSLL765-g/Rq9fFmKD3PI/...         1208
[1208]
                    id                                                url  landmark_id
1215  ab10a5eaeaaa9527  http://lh4.ggpht.com/-aGw2dPaXhaM/SF9CATE_2kI/...          532
[532]
                   id                                                url  landmark_id
568  18e49b065fdd83e0  https://lh5.googleusercontent.com/-HbaeLqLHd6Q...        11107
[11107]
                    id                                                url  landmark_id
2421  deaeaabb1b39fa63  https://lh4.googleusercontent.com/-QRv9f4B6gfU...         8218
[8218]
                    id                                                url  landmark_id
1377  8919

In [99]:
Y.shape

[]

In [2]:
from keras.preprocessing import sequence, text, image                  
from tqdm import tqdm

img_shape = (150, 150) # Image shape (height, width)

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=img_shape)
    
    # convert PIL.Image.Image type to 3D tensor with shape (192, 256, 3)
    x = image.img_to_array(img)
    
    # convert 3D tensor to 4D tensor with shape (1, 192, 256, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [430]:
dira = 'output/'
import random
filename = random.choice(os.listdir(dira))
path = os.path.join(dira, filename)
path

'output/13ae8277b66ccaa8.jpg'

In [436]:
for imgs in os.listdir('output/'):
    try:
        a = path_to_tensor('output/' + imgs)
    except:
        pass

In [72]:
X, Y = create_x_y('output/')
Y = pd.get_dummies(np.squeeze(Y))

NameError: name 'create_x_y' is not defined

In [63]:
X_train.shape

NameError: name 'X_train' is not defined

In [367]:
X[0].shape

(150, 150, 3)

In [370]:
X = np.asarray(X)[np.newaxis, :]

In [371]:
X.shape

(1, 2780)

In [374]:
Y.shape

(2780, 1479)

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(np.squeeze(X), Y, test_size=0.2, random_state=1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


NameError: name 'X' is not defined

In [5]:
from keras import Sequential, optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization, Activation, Conv2D, Dense, Dropout, Flatten, GlobalMaxPool2D, MaxPooling2D, Softmax

In [70]:
input_shape = X_train[0].shape[1], X_train[0].shape[2], X_train[0].shape[3] 

NameError: name 'X_train' is not defined

In [69]:
input_shape = X_train[0].shape, X_train.shape[0]. Y_train.shape[1], Y_train.shape[1]

NameError: name 'X_train' is not defined

In [68]:
input_shape

NameError: name 'input_shape' is not defined

In [67]:
modelBN = Sequential()
modelBN.add(Conv2D(filters=16, kernel_size=4, padding='same', use_bias=True, input_shape=input_shape))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=32, kernel_size=3, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=64, kernel_size=3, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=128, kernel_size=2, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Flatten())
modelBN.add(Dense(512, activation='relu'))
modelBN.add(Dropout(0.3))
modelBN.add(Dense(train_data['landmark_id'].nunique(), activation='softmax'))

modelBN.summary()


NameError: name 'input_shape' is not defined

In [383]:
adam = optimizers.Adam(lr=0.02)
modelBN.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])


In [386]:
X_train = X_train[np.newaxis, :]


In [388]:
X_test = X_test[np.newaxis, :]


In [390]:
X_train.shape

(1, 1668)

In [393]:
type(Y_train.to_numpy())

numpy.ndarray

In [15]:
import tensorflow as tf
tf.convert_to_tensor(Y_train)

NameError: name 'Y_train' is not defined

In [66]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_BatchNorm.hdf5', 
                               verbose=1, save_best_only=True)

epochs_batch = 20
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_tet).astype('float32')

hist_BN = modelBN.fit(np.squeeze(X_train), Y_train), validation_data=(np.squeeze(X_test), Y_test), 
epochs=epochs_batch, batch_size=64, callbacks=[checkpointer], verbose=1


hist_BN = modelBN.fit(np.squeeze(X_train), Y_train, 
      validation_data=(np.squeeze(X_test), Y_test),
      epochs=epochs_batch, batch_size=64, callbacks=[checkpointer], verbose=1)

hist_BN = modelBN.fit(tf.convert_to_tensor(X_train), Y_train, 
      validation_data=(tf.convert_to_tensor(X_test), Y_test),
      epochs=epochs_batch, batch_size=64, callbacks=[checkpointer], verbose=1)



SyntaxError: can't assign to function call (<ipython-input-66-e68900def27f>, line 8)

In [ ]:
modelBN.load_weights('saved_models/weights.best.from_BatchNorm.hdf5')

landmark_pred = [np.argmax(modelBN.predict(np.expand_dims(tensor, axis=0))) for tensor in X_val]
test_accuracy = np.sum(np.array(landmark_pred) == np.argmax(Y_val, axis=1)) / len(landmark_pred)
print('Test accuracy:', test_accuracy)


In [ ]:
import albumentations

In [ ]:
transforms_672 = albumentations.Compose([
    albumentations.Resize(672, 672),
    albumentations.Normalize()
])

transforms_768 = albumentations.Compose([
    albumentations.Resize(768, 768),
    albumentations.Normalize()
])
transforms_512 = albumentations.Compose([
    albumentations.Resize(512, 512),
    albumentations.Normalize()
])

import cv2
for row in os.listdir('outputs'):
    image = cv2.imread(row)
    image = image[:, :, ::-1]

    res0 = self.transform672(image=image)
    image0 = res0['image'].astype(np.float32)
    image0 = image0.transpose(2, 0, 1)        

    res1 = self.transform768(image=image)
    image1 = res1['image'].astype(np.float32)
    image1 = image1.transpose(2, 0, 1)    

    res3 = self.transform512(image=image)
    image3 = res3['image'].astype(np.float32)        
    image3 = image3.transpose(2, 0, 1)   
